In [1]:
import pandas as pd

# Charger le dataset généré précédemment
df = pd.read_csv('property_global_data_wei.csv')

# Afficher les 5 premières lignes
print("--- Aperçu des données ---")
display(df.head())

# Vérifier les types de données (très important pour le Wei)
print("\n--- Types des colonnes ---")
print(df.dtypes)

# Vérifier s'il y a des valeurs manquantes
print("\n--- Valeurs manquantes ---")
print(df.isnull().sum())

--- Aperçu des données ---


,city,country,longitude,latitude,sqm,total_rooms,typeOfRental,nombre_etoiles,rental_rent
0,Paris,France,2.295347,48.893429,294,8,MONTHLY,1,1616632307980210432
1,Tangier,Morocco,-5.896887,35.713376,163,6,MONTHLY,4,390820201531620352
2,Dubai,UAE,55.201680,25.135395,275,8,MONTHLY,5,3101124939995277824
3,Casablanca,Morocco,-7.662749,33.577634,301,8,MONTHLY,4,520791736013653568
4,New York,USA,-74.043325,40.645410,255,7,MONTHLY,3,2612595995967865856



--- Types des colonnes ---
city               object
country            object
longitude         float64
latitude          float64
sqm                 int64
total_rooms         int64
typeOfRental       object
nombre_etoiles      int64
rental_rent         int64
dtype: object

--- Valeurs manquantes ---
city              0
country           0
longitude         0
latitude          0
sqm               0
total_rooms       0
typeOfRental      0
nombre_etoiles    0
rental_rent       0
dtype: int64


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# 1. Sélection des colonnes utiles
# On retire 'typeOfRental' car il est identique pour tout le monde (MONTHLY)
X = df.drop(columns=['rental_rent', 'typeOfRental'])
y = df['rental_rent']

# 2. Définition des transformations
# - Pour le texte : OneHotEncoder (crée des colonnes 0/1 pour chaque ville/pays)
# - Pour les nombres : StandardScaler (centre les données autour de 0)
categorical_features = ['city', 'country']
numerical_features = ['longitude', 'latitude', 'sqm', 'total_rooms', 'nombre_etoiles']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 3. Application du preprocessing
X_processed = preprocessor.fit_transform(X)

# 4. Division des données (Train/Test split)
# On garde 20% des données pour tester la précision finale
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

print("--- Preprocessing Terminé ---")
print(f"Nombre de caractéristiques après encodage : {X_processed.shape[1]}")
print(f"Lignes pour l'entraînement : {X_train.shape[0]}")

--- Preprocessing Terminé ---
Nombre de caractéristiques après encodage : 28
Lignes pour l'entraînement : 2400


In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# 1. Création du modèle
# n_estimators=100 signifie qu'on utilise 100 arbres de décision différents
model = RandomForestRegressor(n_estimators=100, random_state=42)

# 2. Entraînement
print("Entraînement en cours...")
model.fit(X_train, y_train)

# 3. Prédiction sur le jeu de test
y_pred = model.predict(X_test)

# 4. Évaluation des performances
mae_wei = mean_absolute_error(y_test, y_pred)
mae_eth = mae_wei / 10**18  # Conversion pour que ce soit lisible
r2 = r2_score(y_test, y_pred)

print("\n--- Performances du Modèle ---")
print(f"R² Score (Précision) : {r2:.4f}") # Plus proche de 1, mieux c'est
print(f"Erreur Moyenne (MAE) : {mae_eth:.6f} ETH")

Entraînement en cours...

--- Performances du Modèle ---
R² Score (Précision) : 0.9634
Erreur Moyenne (MAE) : 0.114805 ETH


In [4]:
import joblib

# Sauvegarder le modèle et le préprocesseur (très important de garder les deux !)
joblib.dump(model, 'property_price_model.joblib')
joblib.dump(preprocessor, 'preprocessor.joblib')

print("Modèle et Préprocesseur sauvegardés avec succès !")

Modèle et Préprocesseur sauvegardés avec succès !


In [5]:
# 1. Récupérer les noms des nouvelles colonnes créées par l'encodage des villes/pays
cat_encoder = preprocessor.named_transformers_['cat']
cat_features_transformed = cat_encoder.get_feature_names_out(categorical_features)

# 2. Combiner les noms des colonnes numériques et des nouvelles colonnes catégorielles
all_feature_names = numerical_features + list(cat_features_transformed)

# 3. Créer un DataFrame avec X_processed (on convertit en tableau dense si nécessaire)
# Si X_processed est une matrice sparse, on utilise .toarray()
import numpy as np
import scipy.sparse
if scipy.sparse.issparse(X_processed):
    X_final_view = pd.DataFrame(X_processed.toarray(), columns=all_feature_names)
else:
    X_final_view = pd.DataFrame(X_processed, columns=all_feature_names)

# 4. Afficher le résultat
print("--- APERÇU DES DONNÉES PRÊTES POUR LE MODÈLE ---")
X_final_view.head()

--- APERÇU DES DONNÉES PRÊTES POUR LE MODÈLE ---


,longitude,latitude,sqm,total_rooms,nombre_etoiles,city_Abu Dhabi,city_Barcelona,city_Casablanca,city_Chicago,city_Dubai,...,city_Paris,city_Rabat,city_Sevilla,city_Tangier,city_Valencia,country_France,country_Morocco,country_Spain,country_UAE,country_USA
0,0.200626,1.751522,1.165294,1.142341,-1.418688,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.027284,-0.005578,-0.212918,0.343222,0.719679,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.320086,-1.415783,0.965401,1.142341,1.432468,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.010081,-0.290305,1.238939,1.142341,0.719679,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-1.414646,0.651936,0.754987,0.742781,0.006890,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
# Création d'une nouvelle donnée (ex: un appartement de 80m2 à Casablanca)
nouvelle_maison = pd.DataFrame({
    'city': ['Casablanca'],
    'country': ['Morocco'],
    'longitude': [-7.5898],
    'latitude': [33.5731],
    'sqm': [80],
    'total_rooms': [3],
    'nombre_etoiles': [4]
})

# 1. Transformer la donnée avec le préprocesseur
maison_ready = preprocessor.transform(nouvelle_maison)

# 2. Prédire le prix
prix_wei = model.predict(maison_ready)[0]
prix_eth = prix_wei / 10**18

print(f"Prix prédit pour cet appartement : {prix_eth:.4f} ETH ({int(prix_wei)} Wei)")

Prix prédit pour cet appartement : 0.3722 ETH (372177862884638720 Wei)
